## Qwen tests

this notebook (a) loads ~20 examples from one of the emotional-span datasets (IEMOCAP, MELD, SEMAINE, RECOLA), (b) queries Qwen-audio with prompts tailored for span-level detection, and (c) collects outputs into JSON.

Second part: does this per speaker in a segment of MSP Podcast audio.

In [ ]:
!pip install -q transformers datasets soundfile accelerate opensmile librosa

In [ ]:
#return to this when running on supercomputer

import torch, json
import soundfile as sf
from datasets import load_dataset
from transformers import AutoProcessor, AutoModelForSeq2SeqLM
import torch

model_id = "Qwen/Qwen2-Audio-7B"

model = AutoModelForSeq2SeqLM.from_pretrained(
    model_id,
    device_map="auto",
    torch_dtype=torch.float16,
    load_in_4bit=True
)

audio analysis: users could provide audio and text instructions for analysis during the interaction;

Options: "iemocap", "meld", "semaine", "recola" (if available on HuggingFace Hub)

In [ ]:
# Example: dataset_name = "iemocap"
dataset_name = "iemocap"

# Load first 20 examples
dataset = load_dataset(dataset_name, split="train[:20]")

In [ ]:
def run_qwen_analysis(audio, transcript, file_id):
    inputs = processor(audio["array"], sampling_rate=audio["sampling_rate"], return_tensors="pt").to(model.device)

    def ask_qwen(prompt):
        generated_ids = model.generate(**inputs, max_new_tokens=512, decoder_input_ids=processor.tokenizer(prompt, return_tensors="pt").input_ids.to(model.device))
        return processor.batch_decode(generated_ids, skip_special_tokens=True)[0]

    fast_half_prompt = (
        "Listen carefully to the second half of the audio (from the midpoint to the end). "
        "Identify what emotions, tone, or expressive changes are most prominent. "
        "Summarize briefly in 1–2 sentences."

    prompt_bme = (
        "Summarize the emotional tone and content at the beginning, middle, and end of the audio. "
        "Respond strictly in JSON with keys 'beginning', 'middle', and 'end', "
        "where each value is a short description of what you hear emotionally and verbally."
    )

    prompt_shift = (
        "Locate the first clear emotional shift in the transcript (e.g., from neutral to angry, "
        "calm to excited, or sad to hopeful). "
        "Return a JSON object with fields: {'shift_time': approximate_second, 'span': transcript_excerpt}. "
        "Keep 'span' to ≤20 words centered around the shift point."
    )

    prompt_intensity = (
        "Identify the time range where the speaker’s emotional intensity increases or decreases significantly. "
        "Return a JSON object with fields: {'start_time': X, 'end_time': Y, 'span': transcript_excerpt}. "
        "Choose a transcript span of ~1–2 sentences that best illustrates the change."
    )

    prompt_expressive = (
        "Find the portion of the transcript where the speaker is most emotionally expressive "
        "(e.g., raised voice, strong affect, laughter, crying). "
        "Mark this portion by wrapping it with [emote/] ... [/emote]. "
        "Return only the modified transcript, with exactly one [emote/] ... [/emote] span."
    )

    return {
        "file": file_id,
        "transcript": transcript,
        "fast_half": ask_qwen(fast_half_prompt),
        "begin_mid_end": ask_qwen(prompt_bme),
        "emotional_shift": ask_qwen(prompt_shift),
        "intensity_span": ask_qwen(prompt_intensity),
        "expressive_span": ask_qwen(prompt_expressive),
    }


In [ ]:
results = []

for i, ex in enumerate(dataset):
    audio = ex["audio"]
    transcript = ex.get("transcription") or ex.get("text") or ""
## un/comment result as needed
    #result = run_mellow_analysis(audio, transcript, f"sample_{i}")
    result = run_qwen_analysis(audio, transcript, f"sample_{i}")
    results.append(result)

with open("span_detection_results.json", "w") as f:
    json.dump(results, f, indent=2)

print("✅ Saved results to iemo_span_detection_results.json"

## MSP Podcast Implementation



Load MSP podcast

In [ ]:
from datasets import load_dataset

msp = load_dataset("msp-podcast", split="train[:50]")  # e.g., first 50 for test

Take valency measure

In [ ]:
import opensmile

smile = opensmile.Smile(
    feature_set=opensmile.FeatureSet.eGeMAPSv02,
    feature_level=opensmile.FeatureLevel.Functionals
)

def extract_valence_arousal(audio_array, sr):
    # compute features
    features = smile.process_signal(audio_array, sr)
    # pick valence/arousal proxies (approximation)
    valence = features['F0semitoneFrom27.5Hz_sma3nz']  # placeholder
    arousal = features['voicingFinalUnclipped_sma3nz']   # placeholder
    return valence.values, arousal.values

def detect_valence_boundaries(valence, arousal, sr, window_sec=2.0, threshold=0.5):
    # compute delta over sliding window
    delta_val = np.diff(valence)
    delta_ar = np.diff(arousal)
    # candidate boundary = large change
    candidate_times = np.where(np.abs(delta_val) > threshold)[0] / sr
    candidate_times = list(candidate_times)
    return candidate_times

In [ ]:
def ask_qwen(audio, transcript, prompt):
    inputs = processor(audio, sampling_rate=16000, return_tensors="pt").to(model.device)
    prompt_ids = processor.tokenizer(prompt, return_tensors="pt").input_ids.to(model.device)
    outputs = model.generate(
        **inputs,
        max_new_tokens=512,
        decoder_input_ids=prompt_ids
    )
    return processor.batch_decode(outputs, skip_special_tokens=True)[0]


prompt_shift = (
    "Locate the first emotional shift in this segment. "
    "Return JSON: {'shift_time': approximate_second, 'span': transcript_excerpt}."
)
prompt_intensity = (
    "Mark the beginning and end of intensity change in this segment. "
    "Return JSON: {'start_time': X, 'end_time': Y, 'span': transcript_excerpt}."
)
prompt_expressive = (
    "Mark the emotionally expressive portion in transcript with [emote/] ... [/emote]. "
    "Return only the modified transcript."
)
prompt_emotion = "For the span tagged [emote/] ... [/emote], what is the dominant emotion? (angry, happy, sad, neutral, disgust, fear, surprise)"

In [ ]:
results = []

for spk_id in set(dataset['speaker_id']):
    # filter speaker turns
    speaker_turns = [ex for ex in dataset if ex['speaker_id']==spk_id][:10]  # 10 turns
    for seg_length in [30, 120]:  # 30 sec and 2 min
        # concatenate audio + transcript until duration reached
        audio_concat = np.concatenate([turn['audio']['array'] for turn in speaker_turns])
        sr = speaker_turns[0]['audio']['sampling_rate']
        if len(audio_concat)/sr > seg_length:
            audio_concat = audio_concat[:seg_length*sr]

        transcript_concat = " ".join([turn.get('transcription') or turn.get('text') for turn in speaker_turns])

        # valence/arousal analysis
        valence, arousal = extract_valence_arousal(audio_concat, sr)
        val_boundaries = detect_valence_boundaries(valence, arousal, sr)

        # Qwen shift / intensity / expressive
        qwen_shift = ask_qwen(audio_concat, transcript_concat, prompt_shift)
        qwen_intensity = ask_qwen(audio_concat, transcript_concat, prompt_intensity)
        qwen_expressive = ask_qwen(audio_concat, transcript_concat, prompt_expressive)
        qwen_emotion = ask_qwen(audio_concat, qwen_expressive, prompt_emotion)

        results.append({
            "speaker_id": spk_id,
            "segment_duration": f"{seg_length}s",
            "valence_boundaries": val_boundaries,
            "qwen_shift": qwen_shift,
            "qwen_intensity": qwen_intensity,
            "expressive_span": qwen_expressive,
            "emotion_label": qwen_emotion
        })

with open("msp_span_detection.json", "w") as f:
    json.dump(results, f, indent=2)

print("✅ Saved results to msp_span_detection.json")